In [1]:
from causaldag import gspo
import causaldag as cd
import random
import numpy as np
from causaldag import MemoizedCI_Tester, partial_correlation_test, partial_correlation_suffstat, msep_test

/Users/chandlersquires/Documents/projects/other/causaldag_examples/venv/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Setup graph, samples, and sufficient statistic

In [2]:
nnodes = 5
nodes = set(range(nnodes))
nlatent = 3
exp_nbrs = 3
nsamples = 5000

Sample a random Erdos-Renyi DAG with random weights. Then, take $n$ samples and forget the first `nlatent` values.

In [3]:
random.seed(869674)
np.random.seed(986)

dag = cd.rand.directed_erdos(nnodes+nlatent, exp_nbrs/(nnodes+nlatent/2))
gdag = cd.rand.rand_weights(dag)
all_samples = gdag.sample(nsamples)
observed_samples = all_samples[:, nlatent:]

Compute the sufficient statistics for the conditional independence test you are using. Pass those sufficient statistics and the conditional independence testing function to a `CI_Tester`. In this case, we use partial-correlation based CI testing.

In [4]:
suffstat = partial_correlation_suffstat(observed_samples)
ci_tester = MemoizedCI_Tester(partial_correlation_test, suffstat, alpha=.01)

## Estimate the MAG and compare to the true MAG

Compute the ground-truth MAG.

In [5]:
mag = dag.marginal_mag(set(range(nlatent)), relabel='default')

/Users/chandlersquires/Documents/projects/other/causaldag_examples/venv/lib/python3.9/site-packages/graphical_models/custom_types.py:13: UserWarning: [Need name!] still needs to be tested. If you intend to use this method, please submit a pull request.
  warn(s)


In [6]:
print("Directed edges:", mag.directed)
print("Bidirected edges:", mag.bidirected)

Directed edges: {(1, 2), (0, 4), (3, 4), (1, 0), (3, 2), (1, 4)}
Bidirected edges: {frozenset({0, 2}), frozenset({1, 3}), frozenset({0, 3})}


Estimate a MAG using GSPo.

In [7]:
est_mag = gspo(nodes, ci_tester)

In [8]:
print("Estimated directed edges:", est_mag.directed)
print("Estimated bidirected edges:", est_mag.bidirected)

Estimated directed edges: {(2, 3), (0, 4), (3, 4), (1, 4), (3, 0)}
Estimated bidirected edges: {frozenset({0, 1}), frozenset({1, 3}), frozenset({1, 2})}


In [9]:
est_mag.shd_skeleton(mag)

1

In [10]:
est_mag.markov_equivalent(mag)

False

## Oracle Estimation

We can also use m-separation in the true graph as our CI tester to check consistency of the algorithm.

In [11]:
oracle_ci_tester = MemoizedCI_Tester(msep_test, mag)

In [12]:
est_mag_oracle = gspo(nodes, oracle_ci_tester)

In [13]:
print("Estimated directed edges:", est_mag_oracle.directed)
print("Estimated bidirected edges:", est_mag_oracle.bidirected)

Estimated directed edges: {(1, 2), (0, 4), (3, 4), (3, 1), (2, 0), (1, 4), (3, 0), (1, 0), (3, 2)}
Estimated bidirected edges: set()


In [14]:
est_mag_oracle.markov_equivalent(mag)

True

# Methods

In [18]:
print([m for m in dir(mag) if not m.startswith('_')])

['add_bidirected', 'add_directed', 'add_node', 'add_nodes_from', 'add_undirected', 'ancestor_dict', 'ancestors_of', 'as_hashed', 'bidirected', 'c_components', 'children_of', 'colliders', 'copy', 'descendant_dict', 'descendants_of', 'directed', 'discriminating_paths', 'discriminating_triples', 'district_of', 'fast_markov_equivalent', 'from_amat', 'get_all_mec', 'has_any_edge', 'has_bidirected', 'has_directed', 'has_undirected', 'induced_subgraph', 'is_imap', 'is_maximal', 'is_minimal_imap', 'is_minimal_imap2', 'is_minimal_imap3', 'is_minimal_imap4', 'legitimate_mark_changes', 'markov_blanket_of', 'markov_equivalent', 'msep', 'msep_from_given', 'neighbors_of', 'nnodes', 'nodes', 'num_bidirected', 'num_directed', 'num_edges', 'num_undirected', 'pairwise_markov_statements', 'parents_of', 'remove_bidirected', 'remove_directed', 'remove_edge', 'remove_edges', 'remove_node', 'remove_undirected', 'resolved_quasisinks', 'shd_skeleton', 'skeleton', 'spouses_of', 'to_amat', 'to_maximal', 'to_pag'

In [19]:
print(mag.to_amat())

[[0. 3. 2. 2. 2.]
 [2. 0. 2. 2. 2.]
 [2. 3. 0. 3. 0.]
 [2. 2. 2. 0. 2.]
 [3. 3. 0. 3. 0.]]


In [20]:
print("Directed edges:", mag.directed)
print("Bidirected edges:", mag.bidirected)

Directed edges: {(1, 2), (0, 4), (3, 4), (1, 0), (3, 2), (1, 4)}
Bidirected edges: {frozenset({0, 2}), frozenset({1, 3}), frozenset({0, 3})}


Observe that A_{ij} = 2 if there is an arrowhead at j on the edge i-j, and A_{ij} = 3 if there is a tail.

For example, we have 1->0, and thus A_{10} = 2 while A_{01} = 3. Meanwhile, we have 0 <-> 2, thus A_{02} = A_{20} = 2.